In [7]:
import os
import json
import base64
import re
from pathlib import Path
from langchain_openai import ChatOpenAI

# === LangChain Model Setup ===
llm = ChatOpenAI(
    base_url="https://api.avalai.ir/v1",
    model="gpt-4o",
    api_key="aa-IbxIACL4oknjL1lneG03Cgum5IrWc0PGV5KyH8JwU3At7yj3",
    temperature=0,
)

# === Helper Functions ===
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")

def invoke_llm_with_image(test_image_path, example_image_path):
    test_base64 = encode_image(test_image_path)
    example_base64 = encode_image(example_image_path)
    test_ext = Path(test_image_path).suffix[1:]
    example_ext = Path(example_image_path).suffix[1:]  # ✅ fixed

    messages = [
        {
            "role": "system",
            "content": "You are a helpful assistant.",
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": 
                    """
🎯 **Task:**  
Count the total number of circles in the image.  
The image includes visible horizontal and vertical rulers (with coordinate labels) to help you navigate the image.

You should use these rulers to **systematically scan the image**, moving left to right, top to bottom, based on the coordinate ticks.

---

🧭 **Scan Strategy:**

1. Start at the top-left corner of the image (e.g., coordinate (0, 0)).
2. Move left to right along the x-axis, using the tick marks as column boundaries (e.g., every 70 pixels).
3. Once a row is done, move down to the next row along the y-axis (e.g., every 100 pixels).
4. Repeat until the entire image has been scanned.

---

🧠 **Rules for Circle Counting:**

- **Majority Area Rule:**  
If a circle overlaps multiple regions, count it only once — in the region where most of its area lies.
- **No Double Counting:**  
Never count a circle more than once.
- **Multiple Circles:**  
The image may contain multiple circles — some regions might have more than one.

---

📋 **Output Format:**  
Simply return:

**Total Number of Circles:** _N_

---

🖼️ **Example Image:**  
The image below shows an example with coordinate rulers and some circles. Analyze it and provide the total number of circles.
                    """
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/{example_ext};base64,{example_base64}",
                        "detail": "auto",
                    },
                },
                {
                    "type": "text",
                    "text": 
                    """
✅ **Example Output:**

**Total Number of Circles:** 12

---

Now analyze the following test image in the same way and provide only the total number of circles:
                    """
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/{test_ext};base64,{test_base64}",
                        "detail": "auto",
                    },
                },
                {
                    "type": "text",
                    "text": "Please count the total number of circles in the image using the coordinate rulers as a guide. Only return the final total count."
                },
            ],
        }
    ]

    ai_response = llm.invoke(messages)
    return ai_response.content

def extract_total_count(text):
    match = re.search(r"\*\*Total Number of Circles:\*\*\s*([0-9]+)", text)
    if match:
        return int(match.group(1))
    return None

# === Main Processing ===
input_root = "output_images"
example_image_path = "output_images/12/trial-12_18.png"
results = []
correct = 0
total = 0

for folder_name in os.listdir(input_root):
    folder_path = os.path.join(input_root, folder_name)
    if not os.path.isdir(folder_path):
        continue

    try:
        true_count = int(folder_name)
    except ValueError:
        print(f"Skipping folder {folder_name}, not a number.")
        continue

    image_files = sorted([
        f for f in os.listdir(folder_path)
        if f.lower().endswith((".png", ".jpg", ".jpeg"))
    ])[:10]  # ✅ Only take first 10 images

    for filename in image_files:
        image_path = os.path.join(folder_path, filename)
        print(f"📷 Processing: {image_path}")

        try:
            response_text = invoke_llm_with_image(image_path, example_image_path)
            predicted_count = extract_total_count(response_text)

            print(f"🔎 Predicted: {predicted_count} | Expected: {true_count}")

            correct_prediction = predicted_count == true_count
            result = {
                "image": image_path,
                "true_count": true_count,
                "predicted_count": predicted_count,
                "correct": correct_prediction,
                "raw_response": response_text
            }

            results.append(result)
            total += 1
            if correct_prediction:
                correct += 1

        except Exception as e:
            print(f"❌ Error processing {image_path}: {e}")

# === Save and Report ===
accuracy = correct / total if total > 0 else 0.0
summary = {
    "accuracy": accuracy,
    "total": total,
    "correct": correct,
    "results": results
}

with open("gpt4o_counting_accuracy_report.json", "w", encoding="utf-8") as f:
    json.dump(summary, f, indent=2, ensure_ascii=False)

print(f"\n✅ Done! Accuracy: {accuracy:.2%} ({correct}/{total})")


📷 Processing: output_images\11\trial-11_0.png
🔎 Predicted: 13 | Expected: 11
📷 Processing: output_images\11\trial-11_12.png
🔎 Predicted: 14 | Expected: 11
📷 Processing: output_images\11\trial-11_14.png
🔎 Predicted: 13 | Expected: 11
📷 Processing: output_images\11\trial-11_17.png
🔎 Predicted: 11 | Expected: 11
📷 Processing: output_images\11\trial-11_19.png
🔎 Predicted: 13 | Expected: 11
📷 Processing: output_images\11\trial-11_20.png
🔎 Predicted: 13 | Expected: 11
📷 Processing: output_images\11\trial-11_22.png
🔎 Predicted: 15 | Expected: 11
📷 Processing: output_images\11\trial-11_28.png
🔎 Predicted: 14 | Expected: 11
📷 Processing: output_images\11\trial-11_29.png
🔎 Predicted: 13 | Expected: 11
📷 Processing: output_images\11\trial-11_32.png
🔎 Predicted: 13 | Expected: 11
📷 Processing: output_images\12\trial-12_11.png
🔎 Predicted: 14 | Expected: 12
📷 Processing: output_images\12\trial-12_15.png
🔎 Predicted: 14 | Expected: 12
📷 Processing: output_images\12\trial-12_16.png
🔎 Predicted: 13 | 